# 

In [26]:
import openai
from openai import OpenAI

# Point to the server
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

In [48]:
# Sentences to classify
sentences = [
    "I hate you and I hope you fail.",
    "What a beautiful day to go for a walk!",
    "Your idea is stupid and nobody cares."
]

sentences = [
    "#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA Democrats Support Antifa, Muslim Brotherhood, MS13, ISIS, Pedophilia, Child Trafficking, Taxpayer Funded Abortion’s, Election Fraud, Sedition And Treason!!! #LockThemAllUp #WWG1WGA #QAnon @USER URL",
    "#ConstitutionDay is revered by Conservatives, hated by Progressives/Socialist/Democrats that want to change it.",
]

# Build a single prompt
prompt = "Classify each of the following sentences as 'hate' or 'non-hate':\n\n"
for i, s in enumerate(sentences, 1):
    prompt += f"{i}. {s}\n"

prompt += "\nReturn the results in the format:\n<number>. <label>\n"

# Make one request for all sentences
response = client.completions.create(
    model="local model",  # currently unused
    prompt=prompt,
    max_tokens=50,
    temperature=0,
    stop=["Classify", "\n\n"]
)

# Print the raw model output
print(response.choices[0].text.strip())

In [28]:
from sklearn.metrics import classification_report, confusion_matrix

In [29]:
import pandas as pd

# evaluation data
# 15923,"#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA Democrats Support Antifa, Muslim Brotherhood, MS13, ISIS, Pedophilia, Child Trafficking, Taxpayer Funded Abortion’s, Election Fraud, Sedition And Treason!!! #LockThemAllUp #WWG1WGA #QAnon ⁦@USER URL",1
#data = pd.read_csv("Subjectivity_mining_assignment_4_5_data/olid-test.csv")


train_df = pd.read_csv("Subjectivity_mining_assignment_4_5_data/olid-train-small.csv")
test_df  = pd.read_csv("Subjectivity_mining_assignment_4_5_data/olid-test.csv")

In [33]:
fewshot_examples = train_df.sample(n=10, random_state=42)

In [34]:
print(fewshot_examples)

          id                                               text  labels
12387  27650  @USER @USER Why is John Kerry running his mout...       0
385    52965        @USER @USER Gun control anyone? #DisarmHate       0
4896   87438                      @USER What manga you reading?       0
3971   67140  @USER You are right. Victoria is on the revers...       0
10394  43102  @USER @USER Thank you @USER . America is respe...       0
12889  25562  @USER They are Antifa democrats disguised as G...       0
4504   27304  @USER @USER @USER How do you come up with all ...       1
3268   56638    @USER bitch my life was flashing before my eyes       1
2439   49314                      @USER Lmao what a joke he is.       0
4688   42416  @USER @USER @USER @USER @USER Looking forward ...       0


In [49]:
def evaluate(base_prompt, data_slice):
    ground_truth = []

    prompt = base_prompt + "\nClassify each of the following lines:\n\n"

    for i, (_, row) in enumerate(data_slice.iterrows(), 1):
        ground_truth.append(row['labels'])
        prompt += f"{i}. {row['text']}\n"

    prompt += "\nReturn the results in this format:\n<number>. <label>\n"

    print("----- Prompt Sent -----")
    print(prompt[:800] + ("...\n" if len(prompt) > 800 else ""))

    response = client.completions.create(
        model="local model",
        prompt=prompt,
        max_tokens=400,
        temperature=0,
        stop=["Classify"]
    )

    raw_output = response.choices[0].text.strip()
    lines = [ln for ln in raw_output.split("\n") if ln.strip()]

    print("----- Raw Model Output -----")
    print(lines)

    final_predictions = []
    for ln in lines:
        if ". " in ln:
            label_text = ln.split(". ", 1)[1].strip().lower()
        else:
            print("⚠️ Skipped line:", ln)
            continue

        if label_text == "offensive":
            final_predictions.append(1)
        elif label_text == "non-offensive":
            final_predictions.append(0)
        else:
            print("⚠️ Unexpected label:", label_text)

    return final_predictions, ground_truth

In [11]:
# get 10:20 from data
#data[10:20]

In [50]:
#prompt 5 = few-shot prompting with randomm
prompt5_text = (
    "You are a classifier of hate speech.\n"
    "Below are example tweets with their correct labels:\n\n"
)

for i, row in fewshot_examples.iterrows():
    label = "offensive" if row["labels"] == 1 else "non-offensive"
    prompt5_text += f"\"{row['text']}\" → {label}\n"

prompt5_text += (
    "\nNow classify each of the following new tweets as "
    "'offensive' or 'non-offensive':\n\n"
)

In [42]:
#prompt 6 = few-shot prompting with actual examples from the training data, the small one

prompt6_text = (
    "You are a classifier of hate speech.\n"
    "Below are example tweets with their correct labels:\n\n"
)

examples = [
    ("LIBERALS ARE LIKE MONKEYS THROWING THEIR OWN SHIT AT EVERYTHING", "offensive"),
    ("Smack the shit outta her 😂😂", "offensive"),
    ("HEY BEE ITCH YOU WILL BE INDICTED AND WILL GO TO GITMO", "offensive"),
    ("These mentally deranged Liberals are totally disgusting jerks!", "offensive"),
    ("She is telling to be best to students. Wow ! What a role model she is.", "non-offensive"),
    ("So what I read was gun control doesn’t work and CA politicians are working against Californians sounds about right", "non-offensive"),
    ("He IS Mark Twain in my eyes. Love him.", "non-offensive"),
    ("Nobody interferes and steals elections for liberals in western democracies including America than Soros. Why there is no investigation and are no charges against him? Yet!", "non-offensive"),
]

for text,label in examples:
    prompt6_text += f"\"{text}\" → {label}\n"

prompt6_text += "\nNow classify each of the following new tweets as 'offensive' or 'non-offensive':\n\n"

In [52]:
# Prompts 
!#TODO: changed

prompts = {
    "prompt1": "Classify each of the following sentences as 'offensive' or 'non-offensive':\n\n",
    "prompt2": """You are a classifier of hate speech. The following data contains language that is either non offensive or offensive. 
Please note these are the definitions of offensive/non-offensive so try to stick to these guidelines.
'offensive': A post that contains insults, threats, or profanity directed towards a person or group.
'non-offensive': A post that is neutral, polite, or does not contain any harmful language."""
,
    "prompt3": """You are a classifier of hate speech. The following data contains language that is either non-offensive or offensive. 
    Please note these are the definitions of offensive/non-offensive, so try to stick to these guidelines. 
    'offensive': all types of expression that incite, promote, spread or justify violence, hatred or discrimination against a person or group of persons, or that denigrates them, by reason of their real or attributed personal characteristics or status such as "race", color, language, religion, nationality, national or ethnic origin, age, disability, sex, gender identity and sexual orientation. 
    'non-offensive': Negation of offensive definition."""
    ,
    "prompt4": """You are a classifier of hate speech. The following data contains language that is either harmful or safe.
    Please note these are the definitions of harmful/safe, so try to stick to these guidelines.
    'harmful': Language that is offensive, abusive, or hateful, and that can cause emotional or psychological harm to an individual or group.
    'safe': Language that does not contain offensive or abusive content, and does not cause harm. """,

    "prompt5": prompt5_text,
    "prompt6": prompt6_text
}


#to quickly check results
test_slice = test_df.head(30)

all_predictions = []
all_truth_labels = []

for piece in range(0, len(test_slice), 10):
    #pred, truth = evaluate(prompts["prompt5"], test_slice[piece:piece+10])
    pred, truth = evaluate(prompts["prompt6"], test_slice[piece:piece+10])

    all_predictions.extend(pred)
    all_truth_labels.extend(truth)

from sklearn.metrics import classification_report
print("lens:", len(all_truth_labels), len(all_predictions))
print("truth classes:", set(all_truth_labels))
print("pred  classes:", set(all_predictions))

n = min(len(all_truth_labels), len(all_predictions))
all_truth_labels = all_truth_labels[:n]
all_predictions  = all_predictions[:n]

print(classification_report(
    all_truth_labels,
    all_predictions,
    labels=[0, 1],
    target_names=['non-offensive','offensive'],
    digits=3,
    zero_division=0
))

----- Prompt Sent -----
You are a classifier of hate speech.
Below are example tweets with their correct labels:

"LIBERALS ARE LIKE MONKEYS THROWING THEIR OWN SHIT AT EVERYTHING" → offensive
"Smack the shit outta her 😂😂" → offensive

Now classify each of the following new tweets as 'offensive' or 'non-offensive':

Classify each of the following lines:

1. #WhoIsQ #WheresTheServer #DumpNike #DECLASFISA Democrats Support Antifa, Muslim Brotherhood, MS13, ISIS, Pedophilia, Child Trafficking, Taxpayer Funded Abortion’s, Election Fraud, Sedition And Treason!!! #LockThemAllUp #WWG1WGA #QAnon ⁦@USER URL
2. #ConstitutionDay is revered by Conservatives, hated by Progressives/Socialist/Democrats that want to change it.
3. #FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendment #RNC #USMC #veterans @USER    @USER @USER @USER @USER   #fakereporting #THESWAMP #dnc #liberals @USER @USER #fakeoutrage @USER  First, it reduces the ca URL
4. #Watching #Boomer getting the news that she is still up for parole al

In [14]:
#print(classification_report(all_truth_labels, all_predictions , target_names=['non-hate', 'hate']))

In [39]:
# from sklearn.metrics import classification_report
# import pandas as pd
#
# #! TODO: this is changed, doesnt run correctly,
# # need a way to loop through the prompts and use the labels with the prompt to create a table like above
#
# def run_all_prompts(prompts, data, step=10):
#     results = {}
#
#     for pname, ptext in prompts.items():
#         print(f"\n===== Running {pname} =====")
#
#         all_predictions = []
#         all_truth_labels = []
#
#         for piece in range(0, len(data), step):
#             pred, truth = evaluate(ptext, data[piece:piece+step])
#             all_predictions.extend(pred)
#             all_truth_labels.extend(truth)
#
#
#         report = classification_report(
#             all_truth_labels,
#             all_predictions,
#             target_names=['non-offensive/safe', 'offensive/harmful'],
#             output_dict=True
#         )
#         results[pname] = report
#
#
#         print(pd.DataFrame(report).transpose())
#
#     return results
# results = run_all_prompts(prompts, data)